In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import seaborn as sns 
from collections import defaultdict
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
df = pd.read_csv('../data/final-data.csv', sep='~')
x = range(3,len(df.columns)-1)
df = df.iloc[:,x]
df['rym rating'] = df['rym rating']*20
df['rymspot rate'] = (df['rym rating']*.5 + df['popularity']*.5)
# df['rymspot rate'] = df['popularity']
df['rymspot rate'].describe()

count    7252.000000
mean       56.457750
std         9.763336
min        37.600000
25%        49.300000
50%        56.800000
75%        63.200000
max        85.700000
Name: rymspot rate, dtype: float64

In [26]:
labels = []
# >= 73.675 top .25 of our dataset, 73.675-49.62 middle 50%, < 49.62 bottom 25%
for d in df['rymspot rate']:
    labels+=[0 if d >= 73.675 else 1 if d >= 49.62500 else 2]

In [27]:
x = range(3,len(df.columns)-17)
X = df.iloc[:,x]
X

,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841
...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118


In [28]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
scores = []
for x in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]
    
print(sum(scores)/len(scores))

0.7348833835104646


In [29]:
from sklearn.naive_bayes import GaussianNB
scores = []
for x in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = GaussianNB()
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]
    
print(sum(scores)/len(scores))

0.7089484565998042


In [30]:
from sklearn import tree

scores = []
for i in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                             X, labels, test_size = 0.2)
    for x in range(1,100):
        gnb = tree.DecisionTreeClassifier(random_state = 0, max_depth=i)
        naive = gnb.fit(X_train,y_train)
        y_pred = naive.predict(X_test)

        scores += [accuracy_score(y_test, y_pred)]

    print(sum(scores)/len(scores))

0.7064093728463141
0.7129565816678134
0.7096255456007362
0.7134734665747744
0.7164713990351448
0.7206524236158922
0.7202914246332528
0.7216574776016488
0.7205758480741238
0.7189524465885564
0.7188146106133704
0.7157707328279316
0.7138843238085141
0.7122674017918704
0.7112336319779516
0.7085630599586473
0.704787773138202
0.7023891569032843
0.7001342087126796


In [31]:
from sklearn import tree

scores = []
for x in range(1,500):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = tree.DecisionTreeClassifier(random_state = 0, max_depth=7)
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]

print(sum(scores)/len(scores))

0.7226610353719151


In [32]:
from sklearn.neighbors import KNeighborsClassifier

scores = []
for i in range(18,28):
    X_train, X_test, y_train,y_test =train_test_split(
                                             X, labels, test_size = 0.2)
    for x in range(1,50):
        gnb = KNeighborsClassifier(n_neighbors=i)
        naive = gnb.fit(X_train,y_train)
        y_pred = naive.predict(X_test)

        scores += [accuracy_score(y_test, y_pred)]

    print(sum(scores)/len(scores))
#     19 nn

0.7215713301171606
0.7150241212956574
0.711922811853892
0.7119228118538928
0.71330117160579
0.7122674017918703
0.7133011716057907
0.7157133011716088
0.7153687112336333
0.7136457615437658


In [33]:
from sklearn.neural_network import MLPClassifier
scores = []
for x in range(1,100):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]

print(sum(scores)/len(scores))
#     19 nn

0.7019610300106508


In [34]:
# %pip install surprise

In [35]:
# from surprise import KNNWithMeans
# from surprise import Dataset
# from surprise import accuracy
# from surprise import Reader
# from surprise.model_selection import train_test_split



In [36]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# import seaborn as sns 
# from collections import defaultdict
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# df = pd.read_csv('../data/final-data.csv', sep='~')
# x = range(3,len(df.columns)-2)
# df = df.iloc[:,x]
# df['rym rating'] = df['rym rating']*20
# df['Rating'] = (df['rym rating']*.5 + df['popularity']*.5)
# df['Rating'].describe()
# df = df.iloc[:,[12,13,-1]]
# df
# len(np.unique(df.iloc[:,0]))

In [37]:
# reader = Reader(rating_scale=(1, 100))
# data = Dataset.load_from_df(df,reader)

In [38]:
# trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [39]:
# algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
# algo.fit(trainset)

In [40]:
# test_pred = algo.test(testset)

In [41]:
# test_pred

In [42]:
# print("Item-based Model : Test Set")
# accuracy.rmse(test_pred, verbose=True)

https://www.kaggle.com/code/saurav9786/recommender-system-using-amazon-reviews/notebook 

In [43]:
df = pd.read_csv('../data/final-data.csv', sep='~')
x = range(3,len(df.columns)-1)
df = df.iloc[:,x]
x = list(range(3,len(df.columns)-16))
x+=[14,-1]
X = df.iloc[:,x]
X

,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,sp_genres
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240,https://api.spotify.com/v1/tracks/5sQGo3fB2NoF...,"['cantautor', 'latin alternative', 'nueva canc..."
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231,https://api.spotify.com/v1/tracks/1w6mcMacExya...,"['cantautor', 'latin alternative', 'nueva canc..."
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197,https://api.spotify.com/v1/tracks/33QDwXW1O0hA...,"['cantautor', 'latin alternative', 'nueva canc..."
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376,https://api.spotify.com/v1/tracks/2ja41V8fXBHR...,"['cantautor', 'latin alternative', 'nueva canc..."
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841,https://api.spotify.com/v1/tracks/2hf6vQr1Ogju...,"['cantautor', 'latin alternative', 'nueva canc..."
...,...,...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919,https://api.spotify.com/v1/tracks/7BpqssfH7oRR...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975,https://api.spotify.com/v1/tracks/682TDtMAq9Uc...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995,https://api.spotify.com/v1/tracks/0msOgZw6XpjS...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118,https://api.spotify.com/v1/tracks/2yIywMeXB1A4...,"['classic j-rock', 'j-rock', 'japanese dream p..."


In [44]:
import string
for r in range(len(X['sp_genres'])):
    X.iloc[r,-1] = X.iloc[r,[-1]]['sp_genres'].translate(str.maketrans('', '', string.punctuation))
X



,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,sp_genres
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240,https://api.spotify.com/v1/tracks/5sQGo3fB2NoF...,cantautor latin alternative nueva cancion rock...
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231,https://api.spotify.com/v1/tracks/1w6mcMacExya...,cantautor latin alternative nueva cancion rock...
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197,https://api.spotify.com/v1/tracks/33QDwXW1O0hA...,cantautor latin alternative nueva cancion rock...
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376,https://api.spotify.com/v1/tracks/2ja41V8fXBHR...,cantautor latin alternative nueva cancion rock...
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841,https://api.spotify.com/v1/tracks/2hf6vQr1Ogju...,cantautor latin alternative nueva cancion rock...
...,...,...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919,https://api.spotify.com/v1/tracks/7BpqssfH7oRR...,classic jrock jrock japanese dream pop shibuyakei
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975,https://api.spotify.com/v1/tracks/682TDtMAq9Uc...,classic jrock jrock japanese dream pop shibuyakei
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995,https://api.spotify.com/v1/tracks/0msOgZw6XpjS...,classic jrock jrock japanese dream pop shibuyakei
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118,https://api.spotify.com/v1/tracks/2yIywMeXB1A4...,classic jrock jrock japanese dream pop shibuyakei


In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["sp_genres"])
# print("Count Matrix:", count_matrix.toarray())
# np.shape(count_matrix)
# cosine_sim = cosine_similarity(count_matrix)
arr2 = []
np.asarray(arr2)
arr = count_matrix.toarray()
arr.astype(float)
print(len(arr))
for i in range(len(arr)):
    temp = []
    for x in range(len(X.columns)-2):
        temp += [X.iloc[i,x]]
    arr2 += [arr[i].tolist() + temp]
# len(arr2)
cosine_sim = cosine_similarity(arr2)

7252


In [47]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["sp_genres"])
# print("Count Matrix:", count_matrix.toarray())
# np.shape(count_matrix)
# cosine_sim = cosine_similarity(count_matrix)
arr2 = []
for i in range(7252):
    temp = []
    for x in range(len(X.columns)-2):
        temp += [X.iloc[i,x]]
    arr2 += [ temp]
# len(arr2)
cosine_sim = cosine_similarity(arr2)
print(cosine_sim)

[[1.         0.99873446 0.99733688 ... 0.99988945 0.99653537 0.99625676]
 [0.99873446 1.         0.99973762 ... 0.99895499 0.99943921 0.99927708]
 [0.99733688 0.99973762 1.         ... 0.99766783 0.99993024 0.99984684]
 ...
 [0.99988945 0.99895499 0.99766783 ... 1.         0.99696765 0.99664776]
 [0.99653537 0.99943921 0.99993024 ... 0.99696765 1.         0.99995693]
 [0.99625676 0.99927708 0.99984684 ... 0.99664776 0.99995693 1.        ]]


In [48]:
df.track_href[600]

'https://api.spotify.com/v1/tracks/1c9oVo4jlOb18cIPuqGrwE'

In [56]:
hreflike = 'https://api.spotify.com/v1/tracks/1c9oVo4jlOb18cIPuqGrwE'
def get_index_from_href(href):
    return df.index[df['track_href'] == href].tolist()

idx = get_index_from_href(hreflike)
print(idx)


[600]


In [57]:
similar_songs = list(enumerate(cosine_sim[idx]))

In [58]:
df2 = pd.DataFrame(similar_songs[0][1])

df2.describe()

,0
count,7252.000000
mean,0.994981
std,0.016026
min,0.051363
25%,0.995839
50%,0.999153
75%,0.999843
max,1.000000


In [81]:
topfifteen = np.argsort(similar_songs[0][1])[-2:-17:-1]
topfifteen

array([3194, 3470, 6026, 7225, 6687, 3482, 1861, 1365, 6587,  515, 3038,
       3737, 4739, 7052, 1710])

https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599